In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession with Hive support
spark = SparkSession.builder \
    .appName("TableDemo") \
    .enableHiveSupport() \
    .getOrCreate()

# Read CSV from HDFS
hdfs_path = "/tmp/customers_100.csv"  # Update path as per your HDFS location
df = spark.read.option("header", True).csv(hdfs_path)

# Show Data
df.show(5)


25/02/02 03:09:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+
only showing top 5 rows



In [18]:
spark.sql('show tables').show(truncate=False)

+---------+--------------+-----------+
|namespace|tableName     |isTemporary|
+---------+--------------+-----------+
|         |temp_customers|true       |
+---------+--------------+-----------+



In [13]:
spark.sql('drop table temp_customers_2').show()

++
||
++
++



In [15]:

# ---------------- Step 1: Create a Temporary Table (Session-based) ---------------- #
df.createOrReplaceTempView("temp_customers")

print("### Querying Temporary Table (Exists only in this session) ###")
spark.sql("SELECT * FROM temp_customers LIMIT 5").show()


### Querying Temporary Table (Exists only in this session) ###
+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [17]:

# ---------------- Step 2: Create a Global Temporary Table (Accessible across sessions) ---------------- #
df.createOrReplaceGlobalTempView("global_customers")

print("### Querying Global Temporary Table ###")
spark.sql("SELECT * FROM global_temp.global_customers LIMIT 5").show()


### Querying Global Temporary Table ###
+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [19]:
spark.sql('select * from global_temp.global_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [20]:

# ---------------- Step 3: Create a Persistent Table (Stored in Hive Metastore) ---------------- #
spark.sql("DROP TABLE IF EXISTS customers_persistent")
df.write.mode("overwrite").saveAsTable("customers_persistent")

print("### Querying Persistent Table (Accessible across sessions and applications) ###")
spark.sql("SELECT * FROM customers_persistent LIMIT 5").show()


25/02/02 03:16:05 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


### Querying Persistent Table (Accessible across sessions and applications) ###
+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [23]:
spark.sql('describe extended customers_persistent').show(truncate =False)

+----------------------------+------------------------------------------------------------+-------+
|col_name                    |data_type                                                   |comment|
+----------------------------+------------------------------------------------------------+-------+
|customer_id                 |string                                                      |null   |
|name                        |string                                                      |null   |
|city                        |string                                                      |null   |
|state                       |string                                                      |null   |
|country                     |string                                                      |null   |
|registration_date           |string                                                      |null   |
|is_active                   |string                                                      |null   |


In [28]:
# Create a new session within the same Spark application
spark_new = spark.newSession()

# Verify the view exists
print("### Querying Global Temporary Table ###")
spark_new.sql("SHOW TABLES IN global_temp").show()

# Query the Global Temp View
spark_new.sql("SELECT * FROM global_temp.global_customers").show()


### Querying Global Temporary Table ###
+-----------+----------------+-----------+
|  namespace|       tableName|isTemporary|
+-----------+----------------+-----------+
|global_temp|global_customers|       true|
+-----------+----------------+-----------+

+-----------+-----------+---------+-----------+-------+-----------------+---------+
|customer_id|       name|     city|      state|country|registration_date|is_active|
+-----------+-----------+---------+-----------+-------+-----------------+---------+
|          0| Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1| Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2| Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3| Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4| Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
|          5| Customer_5|Hyderabad|  Karnataka|  India|       2023-07-28

In [29]:
spark_new.sql("SELECT * FROM temp_customers LIMIT 5").show()

AnalysisException: Table or view not found: temp_customers; line 1 pos 14;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Project [*]
      +- 'UnresolvedRelation [temp_customers], [], false


In [32]:
spark.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
|         |      temp_customers|       true|
+---------+--------------------+-----------+



In [33]:
spark.stop()